The goal of this notebook is similar to the previous one -- except that we will see how to call C code from Julia. We'll make use of the function `ccall`. Let's take a look.

(More references in this link: https://docs.julialang.org/en/v1/manual/calling-c-and-fortran-code/index.html)

In [1]:
?ccall

search: ccall AbstractChannel



```
ccall((function_name, library), returntype, (argtype1, ...), argvalue1, ...)
ccall(function_name, returntype, (argtype1, ...), argvalue1, ...)
ccall(function_pointer, returntype, (argtype1, ...), argvalue1, ...)
```

Call a function in a C-exported shared library, specified by the tuple `(function_name, library)`, where each component is either a string or symbol. Instead of specifying a library, one can also use a `function_name` symbol or string, which is resolved in the current process. Alternatively, `ccall` may also be used to call a function pointer `function_pointer`, such as one returned by `dlsym`.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression.

Each `argvalue` to the `ccall` will be converted to the corresponding `argtype`, by automatic insertion of calls to `unsafe_convert(argtype, cconvert(argtype, argvalue))`. (See also the documentation for [`unsafe_convert`](@ref Base.unsafe_convert) and [`cconvert`](@ref Base.cconvert) for further details.) In most cases, this simply results in a call to `convert(argtype, argvalue)`.


In [2]:
t = ccall((:clock, "libc"), Int32, ())

10702181

In [5]:
#ccall((:floor,"math"),Float64,(Float64,),10.1)
ccall((:floor,"libm"),Float64,(Float64,),10.1)

10.0

Similar to the Python scenario, make sure that your C code runs before you try calling it from Julia. There is one additional component to getting your C code working, and that is generating a shared object or dynamic library.

First, we'll cover how to generate a dylib just to show how to call the C function. And then, we will write a make file that will be generic.

In [6]:
;gcc -g -O -c hello_world.c 

In [7]:
;gcc -dynamiclib -o hello_world_lib.dylib hello_world.o

In [8]:
ccall((:hello_world,"hello_world_lib.dylib"),
    Int64,
    (),
    )

Hello, World!


0

In [9]:
ccall((:hello_world_repeated,"hello_world_lib.dylib"),
    Int64,
    (Int64,),
    10)

Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!


0

In [14]:
;make clean

rm -f -rf hello_world.o hello
rm -f -rf /Users/hnassar/Dropbox/juliacon2020/hello_world_lib.dylib hello


In [15]:
;make

gcc -g -O -c hello_world.c
gcc -dynamiclib -o /Users/hnassar/Dropbox/juliacon2020/hello_world_lib.dylib hello_world.o


In [16]:
ccall((:hello_world_repeated,"hello_world_lib.dylib"),
    Int64,
    (Int64,),
    10)

Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!


0

In [19]:
function julia_hello_wrapper(n::Int)
    ccall((:hello_world_repeated,"hello_world_lib"),
    Int64,
    (Int64,),
    n)
end

julia_hello_wrapper (generic function with 1 method)

In [20]:
julia_hello_wrapper(10)

Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!


0

More resources
- RCall.jl: https://github.com/JuliaInterop/RCall.jl
- CxxWrap.jl: https://github.com/JuliaInterop/CxxWrap.jl